In [1]:
from scipy import *
from scipy.integrate import quad 
from scipy.linalg import solve
from numpy import *
from tokenize import TokenError
import bqplot.pyplot as plt
from bqplot import *
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr

In [2]:
x = sym.Symbol('x')

## Integral Explorer
Use the entry widgets below the plot to examine different approximations for the integral of the specified function.

The function can be changed by typing a new function into the 'Enter function' box hitting the Return/Enter key.  The function must be described in terms of the variable 'x'. You must use an '\*' to indicate multiplication.  Powers use '\*\*'.

The domain can be changed similarly.

The number of divisions to use in the approximation for the integral can be changed with the labelled slider.

The integration approximation choices are:

| Method    |  Description  |
|-----------|---------------|
| Left      | Rectangular box matching left side |
| Right     | Rectangular box matching right side |
| Trapezoid | Trapezoidal Rule |
| Simpson   | Simpson's Rule |

The "# points for plot" box determines the number of points used to produce the heavy black curve for the function.  200 works well unless there is a rapidly changing function (like sin(1/x) for small x).

Note:  Simpson's Rule will apply a Trapezoid Rule to the last interval if there are an odd number of intervals (*i.e.* an even number of points)

Note 2:  Hovering over a segment will highlight that segment and report its area below the plot

Note 3:  *Clicking* on a segment will bring up a detailed plot of that segment; this will go away if you click on a non-segment part of the plot.


### Try:
(1) Enter the function 'sin(3\*x) \* exp(-x/2)'

(2) Try tanh(x)

(3) Change domain to 0.05 to 3, and function to sin(1/x)

In [5]:
def set_function(strFunc):
    try:
        local_func = lambda x: eval(strFunc)
        def local_deriv(x):
            return around(derivative(local_func, x, dx=1e-6), 4)
        f = parse_expr(strFunc)
    except TokenError:
        print("malformed function")
        return None
    return local_func, local_deriv, f

strFunc = "1+x**2"
f, df, fstr = set_function(strFunc)
sym_func = sym.latex(fstr)

label_layout = widgets.Layout(display='flex', justify_content='center', width = "100%", 
                              align_items = 'center', border = 'none')
fig_layout = widgets.Layout(height = "350px", width = "350px")
fig_margin_dict = dict(top=30, bottom = 30, left = 30, right = 10)

label = widgets.Label(r"$f(x) =  {}$".format(sym.latex(1+x**2)), layout = label_layout)
title_format = "<h2><font color='blue'>$f(x) = {}$</font></h2>"
title_label = widgets.HTMLMath(title_format.format(sym_func),
                              layout = label_layout)

# create initial plots
low = -3
high = 3
xpoint = 0
nPts = 200  # number of points in domain
xx = linspace(low, high, nPts)
yy = f(xx)

# Main figure
fig_color = 'black'
fig_layout = widgets.Layout(height = "400px", width = "100%")
main_fig = plt.figure(title="",  layout = fig_layout, fig_margin = fig_margin_dict)
function_curve = plt.plot(xx, yy, colors = [fig_color], stroke_width = 3)
plot_box = widgets.HBox([main_fig])

xdiv = linspace(low, high, 7)

# define patches for integral estimates:
patch_color = 'LightGreen'
patch = plt.plot([], [],
                fill_colors=[patch_color],
                fill='inside',
                axes_options={'x':{'visible':True, 'y':{'visible':False}}},
                stroke_width = 1,
                interactions={'click':'tooltip'},
                close_path = True,
                opacities = [0.5]) 

def make_div(xdiv, f = f, method = 'Left'):
    tempx = []
    tempy = []
    areas = []
    N = len(xdiv) - 1
    delx = xdiv[1] - xdiv[0]
    for i, x in enumerate(xdiv[:-1]):
        next_x = xdiv[i+1]
        tempx.append([x, x, next_x, next_x])
        y1 = f(next_x) if method == 'Right' else f(x)
        y2 = f(x) if method == 'Left' else f(next_x)
        tempy.append([0, y1, y2, 0])
        areas.append(0.5*(y1+y2))
    patch.x = tempx
    patch.y = tempy
    patch.fill_colors = [patch_color]*N
    patch.colors = ['blue']*N
    patch.opacities = [0.5]*N
    return array(areas)*delx

def make_parabola(xpoints, ypoints):
    A = []
    for x in xpoints:
        A.append([x**2, x, 1])
    coef = solve(array(A), ypoints)
    return lambda x: coef[0]*x**2 + coef[1]*x + coef[2]

def make_simps(xdiv, f = f, method = 'Simpson', npts=25):
    tempx = []
    tempy = []
    areas = []
    N = len(xdiv)-1
    delx = xdiv[1] - xdiv[0]
    for i in range((len(xdiv)-1)//2):
        xpoints = xdiv[2*i:2*i+3]
        ypoints = f(xpoints)
        f_parab = make_parabola(xpoints, ypoints)
        
        for j in [0, 1]:
            x_curve = linspace(xpoints[j], xpoints[j+1], npts)
            y_curve = f_parab(x_curve)
            tempx.append([xpoints[j]]+list(x_curve)+[xpoints[j+1]])
            tempy.append([0]+list(y_curve)+[0])
            areas.append(quad(f_parab, xpoints[j], xpoints[j+1])[0])
            
    # if even number of points, add trapezoid patch as last patch
    # note:  using ragged list of lists is deprecated
    if len(xdiv)%2 == 0:
        x = xdiv[-2]
        next_x = xdiv[-1]
        y1 = f(x)
        y2 = f(next_x)
        tempx.append([x, x, next_x, next_x] + [nan]*(npts-2))
        tempy.append([0, y1, y2, 0]+ [nan]*(npts-2))
        areas.append(0.5*(y1+y2)*(next_x-x))
        
    patch.x = tempx
    patch.y = tempy
    patch.fill_colors = [patch_color]*N
    patch.colors = ['blue']*N
    patch.opacities = [0.5]*N
    return array(areas)
                         
# number of division chooser
N_slider = widgets.IntSlider(value = 7, min = 1, max = 100, step = 1, readout = True,
                            style={'handle_color':'lightblue'})
N_label = widgets.Label("Choose number of divisions: ")

method_layout = widgets.Layout(width = '100px')
method_choice = widgets.Dropdown(options = ['Left', 'Right', 'Trapezoid', 'Simpson'],
                                value = 'Left', layout = method_layout)
method_label = widgets.Label("Choose integration type: ")
N_box = widgets.HBox([N_label, N_slider, method_label, method_choice])
 
# function entry
func_margin_dict = dict(left = 10, right = 10)
func_layout = widgets.Layout(width = '75px')
func_label = widgets.Label("Enter function: ")
func_entry = widgets.Text(strFunc, continuous_update = False)
Npt_label = widgets.Label("# points for plot: ", layout = widgets.Layout(margin = "0 0 0 40px"))
Npt_entry = widgets.BoundedIntText(value = 200, min = 50, max = 1000, step = 50, layout = func_layout)
func_box = widgets.HBox([func_label, func_entry, Npt_label, Npt_entry])

# domain entry
dom_layout = widgets.Layout(width = '75px')
dom_label = widgets.Label("Enter domain: ")
dom_low = widgets.FloatText(value = -3.0, layout = dom_layout)
dom_high = widgets.FloatText(value = 3.0, layout = dom_layout)
out_layout = widgets.Layout(border="", width = '50%', margin = "0 0 0 40px")
out_str = r"<b>Integral Values:  <font color='green'>Approx = {:.4f}</font>, True = {:.4f}</b>"
output = widgets.HTML(value = out_str.format(5.0, 5.0), layout = out_layout)
dom_box = widgets.HBox([dom_label, dom_low, dom_high, output])

# lower line
segment_out_format = r"<b>Segment Area: <font color='blue'>{:.6f}</b>"
segment_blank = "No segment selected yet"
segment_out = widgets.HTML(value = segment_blank)

# callback function
areas = zeros(nPts)
def update_plot(change):
    global areas, f
    strFunc = func_entry.value
    f, df, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    low = dom_low.value
    high = dom_high.value
    true_value = quad(f, low, high)[0]
    
    nPts = Npt_entry.value
    x = linspace(dom_low.value, dom_high.value, nPts)
    y = f(x)
    
    # update main plot
    function_curve.x = x
    function_curve.y = y
    
    # reset segment area
    segment_out.value = segment_blank
    
    N = N_slider.value
    xdiv = linspace(low, high, N)
    delx = xdiv[1] - xdiv[0]
    method = method_choice.value
    div_create = make_simps if method == 'Simpson' else make_div
    areas = div_create(xdiv, f, method)
    area = sum(areas)
    output.value = out_str.format(area, true_value)
    
for widget in [func_entry, dom_low, dom_high, method_choice, N_slider, Npt_entry]:
    widget.observe(update_plot, names = 'value')

update_plot('test')

# handler for hovering on segments:
selected_color = 'blue'
def hover_handler(mark, event):
    try:
        idx = event['data']['index']
        temp = [patch_color]*len(patch.x)
        temp[idx] = selected_color
        patch.fill_colors = temp
        widgets.Label("The area is %.4f" % areas[idx])
        segment_out.value = segment_out_format.format(areas[idx])
    except KeyError:
        pass
    
def click_handler(mark, event):
    try:
        idx = event['data']['index']
        x_start = event['data']['values'][0]['x']
        x_end = event['data']['values'][-1]['x']
        x_patch = [item['x'] for item in event['data']['values']]
        y_patch = [item['y'] for item in event['data']['values']]
        x_sc = LinearScale()
        y_sc = LinearScale()
        width = x_end - x_start
        x_begin = x_start - 0.05*width
        x_stop = x_end + 0.05*width
        
        x_data = linspace(x_begin, x_stop, 25)
        y_data = f(x_data)
        lc = Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc}, colors = [fig_color], 
                   stroke_width = 3)
        mini_patch =  Lines(x=x_patch, y=y_patch,
                            scales={'x': x_sc, 'y': y_sc},
                fill_colors=['LightGreen'],
                fill='inside',
                colors = ['blue'],
                stroke_width = 1,
                close_path = True,
                opacities = [0.5],) 
        ax_x = Axis(scale=x_sc)
        ax_y = Axis(scale=y_sc, orientation='vertical')
        
        tooltip_fig = Figure(title="Segment Detail", marks=[lc, mini_patch], axes=[ax_x, ax_y], 
                             layout=widgets.Layout(min_width='600px', min_height='400px'))
        patch.tooltip = tooltip_fig
    except KeyError:
        pass
    
patch.on_hover(hover_handler)
patch.on_element_click(click_handler)

widgets.VBox([title_label, main_fig, N_box, func_box, dom_box, segment_out])

In [4]:
%%html
<style>
div.input{
    display:none;
}
</style>